In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

scilens_dir = str(Path.home()) + '/Dropbox/scilens/'
hn_vocabulary = open(scilens_dir + 'small_files/hn_vocabulary/hn_vocabulary.txt').read().splitlines()

In [2]:
scilens_dir = scilens_dir + '/cache/diffusion_graph/scilens_3M/'
df = pd.read_csv(scilens_dir + 'article_details_v1.tsv.bz2', sep='\t')

In [16]:
df.title.apply(lambda t: len([w for w in hn_vocabulary if w in str(t)])).value_counts()

0    6680
1    4898
2    1039
3     235
4      40
6       5
5       3
Name: title, dtype: int64

In [26]:
df.full_text.apply(lambda t: min(5,len([w for w in hn_vocabulary if w in str(t)]))).value_counts()

5    8966
4     859
1     805
3     795
2     784
0     691
Name: full_text, dtype: int64

In [27]:
df.full_text.apply(lambda t: min(5,len([w for w in hn_vocabulary if w in str(t.split('\n')[0])]))).value_counts()

0    5217
1    3491
2    1992
3    1043
5     655
4     502
Name: full_text, dtype: int64

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
df['title_vector'] = df.title.apply(lambda x: nlp(str(x)).vector)

In [ ]:
from sklearn.manifold import TSNE
tsne_model_en_2d = TSNE(perplexity=30, n_components=2, verbose=1, init='pca', n_iter=3500, random_state=32)

tsne_result = tsne_model_en_2d.fit_transform(np.stack(df['title_vector']))


In [ ]:
tsne_result.shape

In [ ]:
import seaborn as sns

plt.figure(figsize=(16,10))
sns.scatterplot(
    x = tsne_result[:,0],
    y = tsne_result[:,1],
    #hue="y",
    #palette=sns.color_palette("hls", 10),
    #data=tsne_result,
    legend="full",
    #alpha=0.3
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary=hn_vocabulary)

D = vectorizer.fit_transform(list(df.title.apply(str)))

In [ ]:
D.shape

In [ ]:
tsne_result_2 = tsne_model_en_2d.fit_transform(D.toarray())


In [ ]:
import seaborn as sns

plt.figure(figsize=(16,10))
sns.scatterplot(
    x = tsne_result_2[:,0],
    y = tsne_result_2[:,1],
    #hue="y",
    #palette=sns.color_palette("hls", 10),
    #data=tsne_result,
    legend="full",
    #alpha=0.3
)